<a href="https://colab.research.google.com/github/Aleena-Shafiq/Text_to_image/blob/main/Text_to_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Aleena-Shafiq/Text_to_image.git
!pip install -r Text_to_image/backend/requirements.txt
!pip install diffusers["torch"]

Cloning into 'Text_to_image'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 31 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (31/31), 2.21 MiB | 2.85 MiB/s, done.
Resolving deltas: 100% (6/6), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --upgrade --ignore-installed blinker

In [3]:
!pip install Flask --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: Flask
    Found existing installation: Flask 2.1.0
    Uninstalling Flask-2.1.0:
      Successfully uninstalled Flask-2.1.0


In [4]:
!pip install --upgrade diffusers flax jax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.4/244.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.2 MB/s eta 0:00:00
  Attempting uninstall: jax
    Found existing installation: jax 0.4.16
    Uninstalling jax-0.4.16:
      Successfully uninstalled jax-0.4.16
  Attempting uninstall: flax
    Found existing installation: flax 0.5.0
    Uninstalling flax-0.5.0:
      Successfully uninstalled flax-0.5.0


In [ ]:
from threading import Thread, Event

app_port = 8000
announce_url = None
cloudflared_startup = Event()

def update_announce_url(url):
    global announce_url
    announce_url = url

def start_cloudflared(port):
  from flask_cloudflared import _run_cloudflared
  try:
    announce_url = _run_cloudflared(port, 8888)
  except:
    raise
  finally:
    update_announce_url(announce_url)
    cloudflared_startup.set()

def run_with_cloudflared(thread):
    old_run = thread.run

    def new_run(*args, **kwargs):
        new_thread = Thread(target = start_cloudflared, args=(app_port, ))
        new_thread.setDaemon(True)
        new_thread.start()
        old_run(*args, **kwargs)

    thread.run = new_run

def app():
  !python Text_to_image/backend/app.py --port {app_port} --save_to_disk true --img_format jpeg --output_dir generations

if __name__ == '__main__':
    t1 = Thread(target = app)
    run_with_cloudflared(t1)
    t1.start()
    cloudflared_startup.wait()
    print(f"Your url is: {announce_url}")
    t1.join()

<ipython-input-6-7a6ef2784339>:26: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  new_thread.setDaemon(True)


Your url is: https://shown-statistical-treasures-smell.trycloudflare.com
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
--> Starting the image generation server. This might take up to two minutes.
2023-11-04 17:02:29.333217: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-04 17:02:29.333273: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-04 17:02:29.333314: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLA